<a href="https://colab.research.google.com/github/Rohanghotane/AutoML-Pipeline-Builder/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
                                                # MOUNTING TO GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls '/content/drive/My drive/iris.csv'

ls: cannot access '/content/drive/My drive/iris.csv': No such file or directory


In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
                                        #LOADING IRIS DATASET AND JSON CONFIGURATION

In [ ]:
df = pd.read_csv('/content/drive/My Drive/iris.csv')
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:
path = '/content/drive/MyDrive/algoparams_from_ui.json'
with open(path, 'r') as f:
    config = json.load(f)

print(config.keys())

dict_keys(['session_name', 'session_description', 'design_state_data'])


In [ ]:
                                                              #LOGGING SETUP

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
                                                            #FEATURE HANDLING

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
target = config['design_state_data']['target']['target']
prediction = config['design_state_data']['target']['prediction_type']

In [ ]:
features_config = config['design_state_data']['feature_handling']
chosen_attributes = [k for k, v in features_config.items() if v['is_selected'] and k != target]


In [ ]:
numerical_features = []
categorical_features = []
num_impute_strategies = {}

In [ ]:
for feat, details in features_config.items():
    if not details['is_selected'] or feat == target:
        continue
    if details['feature_variable_type'] == 'numerical':
        numerical_features.append(feat)
        strategy = details['feature_details']['impute_with']
        value = details['feature_details']['impute_value']
        if strategy == 'custom':
            num_impute_strategies[feat] = ('constant', value)
        else:
            num_impute_strategies[feat] = ('mean', None)
    elif details['feature_variable_type'] == 'text':
        categorical_features.append(feat)

In [ ]:
                                                              #TRANSFORMERS

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
num_transformers = []
for feat in numerical_features:
    strategy, fill_value = num_impute_strategies[feat]
    imputer = SimpleImputer(strategy=strategy, fill_value=fill_value)
    num_transformers.append((feat, Pipeline([
        ('imputer', imputer),
        ('scaler', StandardScaler())]), [feat]))

cat_transformer = ('cat', Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore')) ]), categorical_features)

In [ ]:
                                                            #FEATURE GENERATION

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
if 'feature_generation' in config['design_state_data']:
    fg = config['design_state_data']['feature_generation']
    if 'linear_interactions' in fg:
        for f1, f2 in fg['linear_interactions']:
            df[f'{f1}_times_{f2}'] = df[f1] * df[f2]
            chosen_attributes.append(f'{f1}_times_{f2}')

In [ ]:
                                                              #FEATURE REDUCTION

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor, ExtraTreesClassifier, RandomForestRegressor, RandomForestClassifier
from sklearn.decomposition import PCA

In [ ]:
reduction_type = config['design_state_data']['feature_reduction']['feature_reduction_method']
reduction_step = ('reduction', 'passthrough')

if reduction_type == 'PCA':
    n_features = int(config['design_state_data']['feature_reduction']['num_of_features_to_keep'])
    reduction_step = ('reduction', PCA(n_components=n_features))
elif reduction_type == 'Tree-based':

    if prediction == 'Regression':
        model = ExtraTreesRegressor(n_estimators=5)
    else:
        model = ExtraTreesClassifier(n_estimators=5)
    reduction_step = ('reduction', SelectFromModel(model, max_features=int(
        config['design_state_data']['feature_reduction']['num_of_features_to_keep']), prefit=False))

In [ ]:
                                                                #GRID SEARCH

In [ ]:
models = config['design_state_data']['algorithms']

for model_key, model_info in models.items():
    if not model_info['is_selected']:
        continue


    if prediction == 'Regression' and 'Regressor' in model_key:
        if model_key == 'RandomForestRegressor':
            model = RandomForestRegressor()
            param_grid = {
                'model__n_estimators': [model_info['min_trees'], model_info['max_trees']],
                'model__max_depth': [model_info['min_depth'], model_info['max_depth']],
                'model__min_samples_leaf': [model_info['min_samples_per_leaf_min_value'], model_info['min_samples_per_leaf_max_value']]
            }
        elif model_key == 'LinearRegression':
            model = LinearRegression()
            param_grid = {}


    elif prediction == 'Classification' and 'Classifier' in model_key:
        if model_key == 'RandomForestClassifier':
            model = RandomForestClassifier()
            param_grid = {
                'model__n_estimators': [model_info['min_trees'], model_info['max_trees']],
                'model__max_depth': [model_info['min_depth'], model_info['max_depth']],
                'model__min_samples_leaf': [model_info['min_samples_per_leaf_min_value'], model_info['min_samples_per_leaf_max_value']]
            }
        elif model_key == 'LogisticRegression':
            model = LogisticRegression(max_iter=1000)
            param_grid = {}

    else:
        continue

In [ ]:
                                                          #BUILDING PIPELINE

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_features),
        cat_transformer
    ])

pipe = Pipeline([
    ('preprocessor', preprocessor),
    reduction_step,
    ('model', model)
])

In [ ]:
                                                            #TRAIN TEST SPLIT

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split

In [ ]:
X = df[chosen_attributes]
Y = df[target]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=10)

In [ ]:
                                                            #GRID SEARCH

In [ ]:
grid = GridSearchCV(pipe, param_grid=param_grid, cv=3)
grid.fit(X_train, Y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['sepal_length',
                                                                          'sepal_width',
                                                                          'petal_length']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['species'])])),
                                       ('reduction',
                                        SelectFromModel(estimator=ExtraTreesRegressor(n_estimators=5),
                                                        max_features=4)),
                                       ('model', RandomForestRegressor())]),
             param_grid={'model__max_depth': [20, 25],
                         'model__min_samples_leaf': [5, 10],
                         'model__n_estimators': [10, 20]})

In [ ]:
y_pred = grid.predict(X_test)

In [ ]:
logging.info(f"Model: {model_key}")
logging.info("Best Params: %s", grid.best_params_)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score

In [ ]:
if prediction == 'Regression':
  logging.info("MSE: %f", mean_squared_error(Y_test, y_pred))
  logging.info("R2 Score: %f", r2_score(Y_test, y_pred))
else:
  logging.info("Accuracy: %f", accuracy_score(Y_test, Y_pred))
  logging.info("F1 Score: %f", f1_score(Y_test, y_pred, average='weighted'))

In [ ]:
logging.info("-" * 30)

In [ ]:
import joblib

joblib.dump(grid.best_estimator_, f"best_model_{model_key}.pkl")


['best_model_neural_network.pkl']

In [ ]:
with open(f"metrics_{model_key}.txt", "w") as f:
    if prediction == 'Regression':
        f.write(f"MSE: {mean_squared_error(Y_test, y_pred)}\n")
        f.write(f"R2 Score: {r2_score(Y_test, y_pred)}\n")
    else:
        f.write(f"Accuracy: {accuracy_score(Y_test, y_pred)}\n")
        f.write(f"F1 Score: {f1_score(Y_test, y_pred, average='weighted')}\n")


In [ ]:
!python run_pipeline.ipynb

python3: can't open file '/content/run_pipeline.ipynb': [Errno 2] No such file or directory


In [ ]:
!jupyter nbconvert run_pipeline.ipynb --to python

[NbConvertApp] WARNING | pattern 'run_pipeline.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
  